In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score, make_scorer, confusion_matrix
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.optimize import minimize
from scipy import stats
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import warnings
from sklearn.linear_model import ElasticNetCV, LassoCV, Lasso, LinearRegression
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
#from catboost import CatBoostRegressor
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import pickle

import torch
import torch.nn as nn
import torch.optim as optim

from fastdtw import fastdtw
from scipy.spatial.distance import euclidean


warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('train.csv').set_index('id')
test = pd.read_csv('test.csv').set_index('id')
train.head()

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
id,,,,,,,,,,,,,,,,,,,,,
00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## EDA
experimentations
|function name|description|
|---|---|
|EDA_1|View range of sii score|
|EDA_2|View Covariance Matrix|
|EDA_3|View Box plot of distributions|
|EDA_4|View Statistics after Groupby age|

In [3]:
def EDA_1(df):
  ## Distribution of sii
  display(train.groupby('sii').min()['PCIAT-PCIAT_Total'])
  display(train.groupby('sii').max()['PCIAT-PCIAT_Total'])
  #sii_bucket = [0,31,50,80,100]

def EDA_2(df):
  ## Correlation
  covariance_matrix = train.select_dtypes(include=['float64','int64']).corr()
  display(covariance_matrix)

def EDA_3(df):
  ## Box Plot showing distribution of columns grouped by age
  vis_col = ['Physical-BMI','Physical-Height','Physical-Weight','Physical-Waist_Circumference','Physical-Diastolic_BP','Physical-HeartRate','Physical-Systolic_BP']
  for col in vis_col:
    plt.figure(figsize=(10, 5))
    sns.boxplot(x='Basic_Demos-Age', y=col, data=df)
    plt.title(f'{col} by sii')
    plt.show()

def EDA_4(df):
  ## group train by age and find the mean and std
  df_grouped = df.select_dtypes(include=['float64','int64']).groupby('Basic_Demos-Age').agg(['mean', 'std'])
  return df_grouped

In [4]:
EDA_4(train)

Basic_Demos-Sex           CGAS-CGAS_Score             \
                           mean       std            mean        std   
Basic_Demos-Age                                                        
5                      0.321429  0.469124       61.732394  11.552313   
6                      0.333333  0.472045       64.695652  11.052825   
7                      0.346330  0.476347       64.870968  11.729774   
8                      0.359184  0.480251       66.087948  10.962000   
9                      0.383298  0.486711       65.724252  11.993069   
10                     0.354762  0.479012       65.511450  11.833368   
11                     0.374251  0.484655       70.420513  67.857321   
12                     0.333333  0.472217       66.201117  11.987531   
13                     0.394068  0.489688       64.720588  12.518840   
14                     0.420000  0.494797       65.426230  11.795025   
15                     0.421348  0.495168       63.184466  12.315101   
16                     0.456954  0.499801       63.378378  11.413406   
17                     0.421053  0.495908       65.903226  13.507387   
18                     0.364865  0.484678       63.410256  11.970837   
19                     0.370370  0.492103       61.250000  14.790199   
20                     0.500000  0.510754       59.363636  12.635448   
21                     0.517241  0.508548       55.846154  10.807761   
22                     0.875000  0.353553       71.500000   4.949747   

                Physical-BMI           Physical-Height            \
                        mean       std            mean       std   
Basic_Demos-Age                                                    
5                  16.751048  2.788413       44.813462  1.926404   
6                  16.468210  2.894492       46.533345  2.732059   
7                  16.813720  3.253863       49.138916  3.110556   
8                  17.705490  3.994144       51.434319  2.917345   
9                  17.874013  3.486458       53.457479  2.752978   
10                 18.827199  4.455868       55.586396  2.990931   
11                 19.827135  4.609669       58.204781  3.343351   
12                 20.672142  4.833600       60.515087  3.365120   
13                 20.949558  5.335717       63.203571  3.563755   
14                 22.244483  5.603994       64.921220  3.327111   
15                 22.895784  6.089276       65.725984  3.560181   
16                 24.355125  6.670572       66.139298  3.731320   
17                 23.388164  4.777030       66.839759  3.965897   
18                 24.447013  5.651076       67.317358  4.156446   
19                 25.908522  5.872248       66.710000  4.034568   
20                 27.655478  6.636444       67.312500  4.260771   
21                 25.980781  6.768879       66.177222  4.327068   
22                 28.777482  7.290927       64.166667  1.125463   

                Physical-Weight             ... PCIAT-PCIAT_Total             \
                           mean        std  ...              mean        std   
Basic_Demos-Age                             ...                                
5                     48.200962  10.569038  ...         13.325000  14.552198   
6                     49.592617  14.131335  ...         15.667897  16.048089   
7                     56.586186  19.125495  ...         19.045455  17.486092   
8                     65.706427  18.675367  ...         20.891496  16.025622   
9                     71.715190  19.889110  ...         24.371345  17.462976   
10                    82.535373  25.689304  ...         28.468852  18.411055   
11                    94.420085  32.956189  ...         32.135514  19.054046   
12                   107.494421  34.407861  ...         36.894737  19.407533   
13                   119.145902  35.643626  ...         39.200000  20.009219   
14                   133.353374  38.111005  ...         39.380282  19.420955   
15                   136.369920  4

## Feature Engineering


|input|output|
|---|---|
|data_df|data_df with new features|

experimentations
|function name|description|
|---|---|
|featureEngineering_1|Dropping all season and predictive columns|
|featureEngineering_2|Applying StandardScaler|
|featureEngineering_3||
|featureEngineering_4||

In [5]:
def featureEngineering_1(df):
  season_cols = [col for col in df.columns if 'Season' in col]
  df = df.drop(season_cols, axis=1)
  pred_cols = ['PCIAT-PCIAT_01','PCIAT-PCIAT_02','PCIAT-PCIAT_03','PCIAT-PCIAT_04','PCIAT-PCIAT_05','PCIAT-PCIAT_06','PCIAT-PCIAT_07','PCIAT-PCIAT_08',
               'PCIAT-PCIAT_09','PCIAT-PCIAT_10','PCIAT-PCIAT_11','PCIAT-PCIAT_12','PCIAT-PCIAT_13','PCIAT-PCIAT_14','PCIAT-PCIAT_15','PCIAT-PCIAT_16',
               'PCIAT-PCIAT_17','PCIAT-PCIAT_18','PCIAT-PCIAT_19','PCIAT-PCIAT_20','SDS-SDS_Total_Raw']
  try:
    df = df.drop(pred_cols, axis=1)
  except:
    print("Predictive Columns do not exist")
  return df

def featureEngineering_2(train,test):
  scaler = StandardScaler()
  train = pd.DataFrame(scaler.fit_transform(train), columns=train.columns)
  test = pd.DataFrame(scaler.fit_transform(test), columns=test.columns)
  return train,test

train = featureEngineering_1(train)
test = featureEngineering_1(test)

Predictive Columns do not exist


In [6]:
print(set(test.columns) - set(train.columns))
print(set(train.columns) - set(test.columns))
test = test.drop('SDS-SDS_Total_Raw', axis=1)

{'SDS-SDS_Total_Raw'}
{'PCIAT-PCIAT_Total', 'sii'}


## Time series


|input|output|
|---|---|
|parquet files|ts_df indexed by user id|

experimentations
|function name|description|
|---|---|
|tsEngineering_1|Feature Engineering to extract ts.describe() stats|
|tsEngineering_2|Perform Autoencoding on Featured Engineered statistics|
|tsEngineering_3|Clustering of ts_data based on DTW distances|
|tsEngineering_4|Mapping of ts_data to external data based on DTW distances|

In [7]:
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def read_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df


class AutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim*3),
            nn.ReLU(),
            nn.Linear(encoding_dim*3, encoding_dim*2),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, input_dim*2),
            nn.ReLU(),
            nn.Linear(input_dim*2, input_dim*3),
            nn.ReLU(),
            nn.Linear(input_dim*3, input_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def perform_autoencoder(df, encoding_dim=50, epochs=50, batch_size=32):
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    
    data_tensor = torch.FloatTensor(df_scaled)
    
    input_dim = data_tensor.shape[1]
    autoencoder = AutoEncoder(input_dim, encoding_dim)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters())
    
    for epoch in range(epochs):
        for i in range(0, len(data_tensor), batch_size):
            batch = data_tensor[i : i + batch_size]
            optimizer.zero_grad()
            reconstructed = autoencoder(batch)
            loss = criterion(reconstructed, batch)
            loss.backward()
            optimizer.step()
            
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}]')
                 
    with torch.no_grad():
        encoded_data = autoencoder.encoder(data_tensor).numpy()
        
    df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i + 1}' for i in range(encoded_data.shape[1])])
    
    return df_encoded

In [21]:
def tsEngineering_1():
    #load_time_series("series_train.parquet").to_csv('train_ts.csv', index=False)
    #load_time_series("series_test.parquet").to_csv('test_ts.csv', index=False)
    df_train = pd.read_csv('train_ts.csv')
    df_test = pd.read_csv('test_ts.csv')

    return df_train, df_test

def tsEngineering_2(df_train, df_test):
    train_ts_encoded = perform_autoencoder(df_train.drop('id', axis=1), encoding_dim=60, epochs=100, batch_size=32)
    test_ts_encoded = perform_autoencoder(df_test.drop('id', axis=1), encoding_dim=60, epochs=100, batch_size=32)

    time_series_cols = train_ts_encoded.columns.tolist()
    train_ts_encoded["id"]=df_train["id"]
    test_ts_encoded['id']=df_test["id"]

    return train_ts_encoded, test_ts_encoded

def tsEngineering_3(num_clusters = 5, dirname = "series_train.parquet"):
    # agglomerative clustering using DTW-python
    
    def read_file(filename, dirname):
        accel =  pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet')).iloc[:5000,1:4].values
        return accel

    def compare_time_series(center, distances, dirname) -> pd.DataFrame:
        ids = os.listdir(dirname)
        a = read_file(ids[center], dirname)

        for fname in ids:
            try:
                b = read_file(fname, dirname)
                distance, path = fastdtw(a, b, dist=euclidean)
                distances[-1].append(distance)
            except:
                distances[-1].append(0)

        return ids, distances

    # Finding the reference clustering center (O(5n) = 1hr15mins)
    centers = [0]
    center = 0
    distances = []
    for i in range(num_clusters-1):
        distances.append([])
        ids, distances = compare_time_series(center, distances, dirname)
        distance_np = np.array(distances)
        distance_np *= (distance_np > distance_np.mean(axis=1)[:,None])
        for j in centers:
            distance_np[:,j] = 0
            center = np.argmax(distance_np.sum(axis=0))
        center = np.argmax(np.array(distances).sum(axis=0))
        centers.append(center)
        print(center)

    # Finding the clustering center (O(n))
    clusters = np.argmin(distances,axis=0)
    centers = list(map(int, centers))
    indexes = [x.split('=')[1] for x in ids]
    classify_train_ts = pd.DataFrame({'id':indexes, 'cluster':clusters})

    # Applying All Images to test data
    test_dirname = 'series_test.parquet'
    test_ids = os.listdir(test_dirname)
    data_list = []
    for center in centers:
        data_list.append(read_file(ids[center], dirname))
    
    clusters = []
    for fname in test_ids:
        distances = []
        for data in data_list:
            try:
                b = read_file(fname, test_dirname)
                distance, path = fastdtw(data[:-1], b, dist=euclidean)
                distances += [distance]
            except:
                distances += [1e99]
        clusters.append(np.argmin(distances))

    indexes = [x.split('=')[1] for x in test_ids]
    classify_test_ts = pd.DataFrame({'id':indexes, 'cluster':clusters})
    return classify_train_ts, classify_test_ts

def tsEngineering_4(dirname = "series_train.parquet"):
    
    def read_file(filename, dirname):
        accel =  pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet')).iloc[:5000,1:4].values
        return accel
    
    def read_reference() -> list:
        # create reference images
        data_list = []
        for file in ['1468360517106-acc-watch_tv.txt','1468367368314-acc-eat_chips.txt','1469751427337-acc-sweep.txt',
                        '1468379023854-acc-brush_teeth.txt','1471982025220-acc-wash_hands.txt',
                        '1468683416878-acc-type_on_keyboard.txt','1471982629473-acc-mop_floor.txt']:
            with open(f'user/{file}','r+') as f:
                data = f.read()
                data_list.append([list(map(float, x.split(',')[1:])) for x in data.split('\n')])
        return data_list
    
    ids = os.listdir(dirname)
    data_list = read_reference()

    clusters = []
    for fname in ids:
        distances = []
        for data in data_list:
            try:
                b = read_file(fname, dirname)
                distance, path = fastdtw(data[:-1], b, dist=euclidean)
                distances += [distance]
            except:
                distances += [1e99]
        clusters.append(np.argmin(distances))

    indexes = [x.split('=')[1] for x in ids]
    classify_ts_df = pd.DataFrame({'id':indexes, 'cluster':clusters})
    return classify_ts_df

In [ ]:
classify_train_ts, classify_test_ts = tsEngineering_3(num_clusters=5, dirname="series_train.parquet")

4
2


In [ ]:
classify_train_ts = tsEngineering_4('series_train.parquet')
classify_test_ts = tsEngineering_4('series_test.parquet')

In [27]:
train = pd.merge(train, classify_train_ts, on='id', how='left')
test = pd.merge(test, classify_test_ts, on='id', how='left')

In [ ]:
# preparing external data for tsEngineering_4()
def read_file(file):
    with open(f'user/{file}','r+') as f:
        data = f.read()
        return [list(map(float, x.split(',')[1:])) for x in data.split('\n')]

def compare_time_series(center, distances) -> pd.DataFrame:
    ids = os.listdir('user')
    a = read_file(ids[center])[:-1]

    for fname in ids:
        try:
            b = read_file(fname)[:-1]
            distance, path = fastdtw(a, b, dist=euclidean)
            distances[-1].append(distance)
        except:
            distances[-1].append(0)

    return ids, distances
'''
# Finding the clustering center (O(5n) = 1hr15mins)
num_clusters = 7
centers = [0]
center = 0
distances = []
for i in range(num_clusters):
    distances.append([])
    ids, distances = compare_time_series(center, distances)
    distance_np = np.array(distances)
    distance_np *= (distance_np > distance_np.mean(axis=1)[:,None])
    for j in centers:
        distance_np[:,j] = 0
    center = np.argmax(distance_np.sum(axis=0))
    centers.append(center)
    print(center)

# Finding the clustering center (O(n))
distances = np.array(distances)
clusters = distances.argmin(axis=0)
for i in list(map(int,centers)):
    print(ids[i])'''

1468360517106-acc-watch_tv.txt
1468367368314-acc-eat_chips.txt
1469751427337-acc-sweep.txt
1468379023854-acc-brush_teeth.txt
1471983192578-acc-brush_teeth.txt
1471982025220-acc-wash_hands.txt
1468683416878-acc-type_on_keyboard.txt
1471982629473-acc-mop_floor.txt
1469751995281-acc-type_on_keyboard.txt


In [87]:
df_train, df_test = tsEngineering_1()
train_ts_encoded, test_ts_encoded = tsEngineering_2(df_train, df_test)
train_w_ts = pd.merge(train.reset_index(), train_ts_encoded, how="left", on='id')
test_w_ts = pd.merge(test.reset_index(), test_ts_encoded, how="left", on='id')

Epoch [10/100], Loss: 0.6958]
Epoch [20/100], Loss: 0.6336]
Epoch [30/100], Loss: 0.6242]
Epoch [40/100], Loss: 0.6203]
Epoch [50/100], Loss: 0.6110]
Epoch [60/100], Loss: 0.6083]
Epoch [70/100], Loss: 0.6013]
Epoch [80/100], Loss: 0.6013]
Epoch [90/100], Loss: 0.6007]
Epoch [100/100], Loss: 0.5969]
Epoch [10/100], Loss: 1.0135]
Epoch [20/100], Loss: 0.6371]
Epoch [30/100], Loss: 0.4271]
Epoch [40/100], Loss: 0.4271]
Epoch [50/100], Loss: 0.4271]
Epoch [60/100], Loss: 0.4271]
Epoch [70/100], Loss: 0.4271]
Epoch [80/100], Loss: 0.4271]
Epoch [90/100], Loss: 0.4271]
Epoch [100/100], Loss: 0.4271]


## Imputation of Missing values


|input|output|
|---|---|
|data_df|data_df without missing values|

experimentations
|function name|description|
|---|---|
|imputation_1|MICE Imputer|
|imputation_2|SimpleImputer|
|imputation_3|Statistical Imputation|
|imputation_4|KNNImputer|

In [88]:
def imputation_1(train,test):
  # Multivariate Imputation with Chain-Equation
  from sklearn.experimental import enable_iterative_imputer
  from sklearn.impute import IterativeImputer
  
  imputer = IterativeImputer(max_iter=10, random_state=0)
  mice_train = train.select_dtypes(include=['float64', 'int64'])
  mice_test = test.select_dtypes(include=['float64', 'int64'])

  mice_train = imputer.fit_transform(mice_train)
  mice_test = imputer.fit_transform(mice_test)
  mice_train_df = pd.DataFrame(mice_train, columns=train.columns)
  mice_test_df = pd.DataFrame(mice_test, columns=test.columns)

  return mice_train_df,mice_test_df

def imputation_2(train,test):
  from sklearn.impute import SimpleImputer

  # Initialize the SimpleImputer
  simple_imputer = SimpleImputer(strategy='mean')  # You can change 'mean' to 'median', 'most_frequent', or 'constant'

  # Perform SimpleImputer imputation
  simple_train = simple_imputer.fit_transform(train.select_dtypes(include=['float64', 'int64']))
  simple_test = simple_imputer.fit_transform(test.select_dtypes(include=['float64', 'int64']))
  simple_train_df = pd.DataFrame(simple_train, columns=train.columns)
  simple_test_df = pd.DataFrame(simple_test, columns=test.columns)
  
  return simple_train_df, simple_test_df

def imputation_3(train,test):
  train_stats = EDA_4(train)

  def statsImpute(row):
    # Find column that has missing value
    missing = row.isnull()
    for missing_col in row[missing].index:
      age = row['Basic_Demos-Age']
      mean = train_stats[missing_col]['mean'].loc[age]
      std = train_stats[missing_col]['std'].loc[age]
      # Fill in value with random value from normal distribution
      row[missing_col] = random.gauss(mean,std)
    return row

  stats_train_df = train.apply(statsImpute, axis=1)
  stats_train_df = stats_train_df.fillna(-1)
  stats_test_df = test.apply(statsImpute, axis=1)
  stats_test_df = stats_test_df.fillna(-1)
  return stats_train_df, stats_test_df


def imputation_4(train,test):
  from sklearn.impute import KNNImputer

  # Initialize the KNNImputer
  knn_imputer = KNNImputer(n_neighbors=5)

  # Perform KNN imputation
  knn_train = knn_imputer.fit_transform(train.select_dtypes(include=['float64', 'int64']))
  knn_test = knn_imputer.fit_transform(test.select_dtypes(include=['float64', 'int64']))
  knn_train_df = pd.DataFrame(knn_train, columns=train.columns)
  knn_test_df = pd.DataFrame(knn_test, columns=test.columns)
  return knn_train_df, knn_test_df

train,test = imputation_3(train,test)

In [ ]:
# Code for finding optimal thresholds copied from: Michael Semenoff
# https://www.kaggle.com/code/michaelsemenoff/cmi-actigraphy-feature-engineering-selection

SEED = 9365
n_splits = 10
optimize_params = False
n_trials = 25 # n_trials for optuna 
voting = True
base_thresholds = [30, 50, 80]

def round_with_thresholds(raw_preds, thresholds):
    return np.where(raw_preds < thresholds[0], int(0),
                    np.where(raw_preds < thresholds[1], int(1),
                             np.where(raw_preds < thresholds[2], int(2), int(3))))

def optimize_thresholds(y_true, raw_preds, start_vals=[0.5, 1.5, 2.5]):
    def fun(thresholds, y_true, raw_preds):
        rounded_preds = round_with_thresholds(raw_preds, thresholds)
        return -cohen_kappa_score(y_true, rounded_preds, weights='quadratic')

    res = minimize(fun, x0=start_vals, args=(y_true, raw_preds), method='Powell')
    assert res.success
    return res.x
def calculate_weights(series):
    # Create bins for the target variable and assign weights based on frequency
    bins = pd.cut(series, bins=10, labels=False)
    weights = bins.value_counts().reset_index()
    weights.columns = ['target_bins', 'count']
    weights['count'] = 1 / weights['count']
    weight_map = weights.set_index('target_bins')['count'].to_dict()
    weights = bins.map(weight_map)
    return weights / weights.mean() 
def cross_validate(model_, data, features, score_col, index_col, cv, sample_weights=False, verbose=False):
    """
    Perform cross-validation with a given model and compute the out-of-fold 
    predictions and Cohen's Kappa score for each fold.

    Returns:
    float: Mean Kappa score across all folds.
    array: Out-of-fold score predictions for the entire dataset.
    """
    kappa_scores = [] 
    oof_score_predictions = np.zeros(len(data))  

    score_to_index_thresholds = base_thresholds  
    thresholds = []
    for fold_idx, (train_idx, val_idx) in enumerate(cv.split(data, data[index_col])):
        X_train, X_val = data[features].iloc[train_idx], data[features].iloc[val_idx]
        y_train_score = data[score_col].iloc[train_idx] 
        y_train_index = data[index_col].iloc[train_idx]
        y_val_score = data[score_col].iloc[val_idx]      
        y_val_index = data[index_col].iloc[val_idx]     
        
        # Train model with sample weights if provided
        if sample_weights:
            weights = calculate_weights(y_train_score)
            model_.fit(X_train, y_train_score, sample_weight=weights)
        else:
            model_.fit(X_train, y_train_score)

        y_pred_train_score = model_.predict(X_train)
        y_pred_val_score = model_.predict(X_val)
        
        oof_score_predictions[val_idx] = y_pred_val_score 

        # Find optimal threshold in sample 
        t_1 = optimize_thresholds(y_train_index, y_pred_train_score, start_vals=base_thresholds)
        thresholds.append(t_1)

        y_pred_val_index = round_with_thresholds(y_pred_val_score, t_1)

        kappa_score = cohen_kappa_score(y_val_index, y_pred_val_index, weights='quadratic')
        kappa_scores.append(kappa_score)
        
        if verbose:
            print(f"Fold {fold_idx}: Optimized Kappa Score = {kappa_score}")
    
    if verbose:
        print(f"## Mean CV Kappa Score: {np.mean(kappa_scores)} ##")
        print(f"## Std CV: {np.std(kappa_scores)}")
    
    return np.mean(kappa_scores), oof_score_predictions, thresholds

def n_cross_validate(model_, data, features, score_col, index_col, cv, seeds, sample_weights=False, verbose=False):
    scores = []
    for seed in seeds:
        cv.random_state=seed
        score, oof, _ = cross_validate(model_, data, features, score_col, index_col, cv, sample_weights=True, verbose=False)
        scores.append(score)
    return score, oof
def objective(trial, model_type, X, features, score_col, index_col, cv, sample_weights=False):
    # Parameter space to explore if model is xgboost
    if model_type == 'xgboost':
        params = {
            'objective': trial.suggest_categorical('objective', ['reg:tweedie', 'reg:pseudohubererror']),
            'random_state': SEED,
            'num_parallel_tree': trial.suggest_int('num_parallel_tree', 2, 30),
            'n_estimators': trial.suggest_int('n_estimators', 100, 300),
            'max_depth': trial.suggest_int('max_depth', 2, 4),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.02, 0.05),
            'subsample': trial.suggest_float('subsample', 0.5, 0.8),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 1e-1),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 1e-1),
        }
        if params['objective'] == 'reg:tweedie':
            params['tweedie_variance_power'] = trial.suggest_float('tweedie_variance_power', 1, 2)
        model = XGBRegressor(**params, use_label_encoder=False)
    
    # Parameter space to explore if model is lightgbm
    elif model_type == 'lightgbm':
        params = {
            'objective': trial.suggest_categorical('objective', ['poisson', 'tweedie', 'regression']),
            'random_state': SEED,
            'verbosity': -1,
            'n_estimators': trial.suggest_int('n_estimators', 100, 300),
            'max_depth': trial.suggest_int('max_depth', 2, 4),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.05),
            'subsample': trial.suggest_float('subsample', 0.5, 0.8),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100)
        }
        if params['objective'] == 'tweedie':
            params['tweedie_variance_power'] = trial.suggest_float('tweedie_variance_power', 1, 2)
        model = LGBMRegressor(**params)
    
    # Parameter space to explore if model is catboost
    '''elif model_type == 'catboost':
        params = {
            'loss_function': trial.suggest_categorical('objective', ['Tweedie:variance_power=1.5', 
                                                                     'Poisson', 'RMSE']),
            'random_state': SEED,
            'iterations': trial.suggest_int('iterations', 100, 300),
            'depth': trial.suggest_int('depth', 2, 4),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.05),
            'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 1e-1),
            'subsample': trial.suggest_float('subsample', 0.5, 0.7),
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
            'random_strength': trial.suggest_float('random_strength', 1e-3, 10.0),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 60),
        }
        model = CatBoostRegressor(**params, verbose=0)
    
    else:
        raise ValueError(f"Unsupported model_type: {model_type}")'''
        
    seeds = [random.randint(1, 10000) for _ in range(20)]

    score, _ = n_cross_validate(model, X, features, score_col, index_col, cv, seeds, sample_weights=True, verbose=True)

    return score

def run_optimization(X, features, score_col, index_col, model_type, n_trials=30, cv=None, sample_weights=False):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, model_type, X, features, score_col, index_col, cv, sample_weights), 
                   n_trials=n_trials)
    
    print(f"Best params for {model_type}: {study.best_params}")
    print(f"Best score: {study.best_value}")
    return study.best_params
# Replace if subsets for features have been selected
exclude = ["PC_9", "PC_12", "Fitness_Endurance-Max_Stage", "Basic_Demos-Sex", 'BMI_mean_norm', "PC_11", "PC_8", "FGC_Zones_min", 'Physical-Systolic_BP',
           "PC_4", "BIA-BIA_FMI", "BIA-BIA_LST", "Physical-Diastolic_BP", 'BIA-BIA_ECW', 'Fitness_Endurance-Time_Mins', 'PAQ_C-PAQ_C_Total', 'PC_10',
           'BIA-BIA_Fat', 'FFM_norm', 'PC_14', 'PC_7']
exclude = []

features = train.columns.tolist()
reduced_features = [f for f in features if f not in exclude]

lgb_features = reduced_features
xgb_features = reduced_features
cat_features = reduced_features
print(len(reduced_features))

# Parameters for LGBM, XGB and CatBoost
lgb_params = {
    'objective': 'poisson', 
    'n_estimators': 295, 
    'max_depth': 4, 
    'learning_rate': 0.04505693066482616, 
    'subsample': 0.6042489155604022, 
    'colsample_bytree': 0.5021876720502726, 
    'min_data_in_leaf': 100
}

xgb_params = {'objective': 'reg:tweedie', 'num_parallel_tree': 12, 'n_estimators': 236, 'max_depth': 3, 'learning_rate': 0.04223740904479563, 'subsample': 0.7157264603586825, 'colsample_bytree': 0.7897918901977528, 'reg_alpha': 0.005335705058190553, 'reg_lambda': 0.0001897435318347022, 'tweedie_variance_power': 1.1393958601390142}

xgb_params_2 = {
    'objective': 'reg:tweedie', 
    'num_parallel_tree': 18, 
    'n_estimators': 175, 
    'max_depth': 3, 
    'learning_rate': 0.032620453423049305, 
    'subsample': 0.6155579670568023, 
    'colsample_bytree': 0.5988773292417443, 
    'reg_alpha': 0.0028895066837627205, 
    'reg_lambda': 0.002232531512636924, 
    'tweedie_variance_power': 1.1708678482038286
}

cat_params = {
    'objective': 'RMSE', 
    'iterations': 238, 
    'depth': 4, 
    'learning_rate': 0.044523361750173816, 
    'l2_leaf_reg': 0.09301285673435761, 
    'subsample': 0.6902492783438681, 
    'bagging_temperature': 0.3007304771330199, 
    'random_strength': 3.562201626987314, 
    'min_data_in_leaf': 60
}

xtrees_params = {
    'n_estimators': 500, 
    'max_depth': 15, 
    'min_samples_leaf': 20, 
    'bootstrap': False
}
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
if optimize_params:
    # LightGBM Optimization
    lgb_params = run_optimization(train, lgb_features, 'PCIAT-PCIAT_Total', 'sii', 'lightgbm', n_trials=n_trials, cv=kf, sample_weights=True)

    # XGBoost Optimization
    xgb_params = run_optimization(train, xgb_features, 'PCIAT-PCIAT_Total', 'sii', 'xgboost', n_trials=n_trials, cv=kf, sample_weights=True)

    # CatBoost Optimization
    #cat_params = run_optimization(train, cat_features, 'PCIAT-PCIAT_Total', 'sii', 'catboost', n_trials=n_trials, cv=kf, sample_weights=True)
# Define models
lgb_model = LGBMRegressor(**lgb_params, random_state=SEED, verbosity=-1)
xgb_model = XGBRegressor(**xgb_params, random_state=SEED, verbosity=0)
xgb_model_2 = XGBRegressor(**xgb_params_2, random_state=SEED, verbosity=0)
#cat_model = CatBoostRegressor(**cat_params, random_state=SEED, verbose=0)
xtrees_model = ExtraTreesRegressor(**xtrees_params, random_state=SEED)


def runModel(train,test):
    # Drop non-float64 and non-int64 columns
    train = train.select_dtypes(include=['float64', 'int64'])

    # Continue with your existing code
    weights = calculate_weights(train['PCIAT-PCIAT_Total'])

    # Cross-validate LGBM model
    print('LGBM model')
    score_lgb, oof_lgb, lgb_thresholds = cross_validate(
        lgb_model, train, lgb_features, 'PCIAT-PCIAT_Total', 'sii', kf, verbose=True, sample_weights=True
    )
    lgb_model.fit(train[lgb_features], train['PCIAT-PCIAT_Total'], sample_weight=weights)
    test_lgb = lgb_model.predict(test[lgb_features])

    # Cross-validate XGBoost model
    print('XGBoost model')
    score_xgb, oof_xgb, xgb_thresholds = cross_validate(
        xgb_model, train, xgb_features, 'PCIAT-PCIAT_Total', 'sii', kf, verbose=True, sample_weights=True
    )
    xgb_model.fit(train[xgb_features], train['PCIAT-PCIAT_Total'], sample_weight=weights)
    test_xgb = xgb_model.predict(test[xgb_features])

    # Cross-validate XGBoost model 2
    print('XGBoost model2')
    score_xgb_2, oof_xgb_2, xgb_2_thresholds = cross_validate(
        xgb_model_2, train, xgb_features, 'PCIAT-PCIAT_Total', 'sii', kf, verbose=True, sample_weights=True
    )
    xgb_model_2.fit(train[xgb_features], train['PCIAT-PCIAT_Total'], sample_weight=weights)
    test_xgb_2 = xgb_model_2.predict(test[xgb_features])

    # Cross-validate ExtraTreesRegressor model
    print('ExtraTreesRegressor model')
    score_xtrees, oof_xtrees, xtrees_thresholds = cross_validate(
        xtrees_model, train, reduced_features, 'PCIAT-PCIAT_Total', 'sii', kf, verbose=True, sample_weights=True
    )
    xtrees_model.fit(train[reduced_features], train['PCIAT-PCIAT_Total'], sample_weight=weights)
    test_xtrees = xtrees_model.predict(test[reduced_features])

    # Print overall mean Kappa score for all models
    print(f'Overall Mean Kappa: {np.mean([score_lgb, score_xgb, score_xtrees])}') # Ensemble score likely higher

    lgb_thresholds_ens = np.mean(np.array(lgb_thresholds), axis=0)
    xgb_thresholds_ens = np.mean(np.array(xgb_thresholds), axis=0)
    xgb_2_thresholds_ens = np.mean(np.array(xgb_2_thresholds), axis=0)
    xtrees_thresholds_ens = np.mean(np.array(xtrees_thresholds), axis=0)
    test_lgb = round_with_thresholds(test_lgb, lgb_thresholds_ens)
    test_xgb = round_with_thresholds(test_xgb, xgb_thresholds_ens)
    test_xgb_2 = round_with_thresholds(test_xgb_2, xgb_2_thresholds_ens)
    test_xtrees = round_with_thresholds(test_xtrees, xtrees_thresholds_ens)

    if voting:
        test_preds = np.array([test_lgb, test_xgb, test_xgb_2, test_xtrees])
        voted_test = stats.mode(test_preds, axis=0).mode.flatten().astype(int)
        final_test = np.round(voted_test).astype(int)
    else:
        test_preds = np.array([test_lgb, test_xgb, test_xgb_2, test_xtrees])
        weighted_test = np.average(test_preds, axis=0, weights=weights)
        final_test = np.round(weighted_test).astype(int)
        
    return final_test

81


In [ ]:
test = runModel(train,test)

## Submission
submission = pd.read_csv("/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv")
submission['sii'] = test
submission.to_csv("submission.csv", index=False)